<a href="https://colab.research.google.com/github/thartm10/BDD_ranked_choice/blob/main/RankedChoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from collections import deque
import pandas as pd
import statistics as stats
import random

# #url = 'https://raw.githubusercontent.com/thartm10/BDD_ranked_choice/main/patient_preference_list.csv'
# url = 'https://raw.githubusercontent.com/thartm10/BDD_ranked_choice/main/patient_preference_list_large.csv'
# df = pd.read_csv(url)

def convert_to_df(file): #Reads in csv file and converts it to a dataframe 
    return pd.read_csv(file)

def get_capacity_and_demand(data): #Get patient and doctor numbers from the number of rows and columns from the csv file; get doctor capacity by rounding up the number patients for doctor
    doctorNum, patientNum = data.shape
    doctorCapacity = (patientNum // doctorNum) + 1
    return doctorCapacity, patientNum



def greedy_rank(data, doctCap, patNum): #Tries to add patients to primary choice in random order, compares the existing patients to see if they ranked the doctor higher and either replaces the exisiting patient or moves on to the next doctor on the list
    df = data
    patientNum = patNum
    doctorCapacity = doctCap


    matches = {} #Maps doctors to patients they have been tentatively matched with 
    for doctor in df.index: #Initializes the matches with empty sets 
        matches[doctor] = set() 
    q = deque() #Queue for the patients before they get assinged to a doctor 

    patientList = list(df) #Gets list of patients from the column names 
    random.shuffle(patientList) #Randomizes the list order for fairness
    for patient in patientList: #Adds each patient in the randomized list to a queue
        q.append(patient)

    while len(q) != 0: #Until there are no patients left 
        tentPatient = q.popleft() #Get the first patient in the list
        noMatch = True #Indicates wether a match has been found to end the loop
        index = 0 #Keeps track of which doctor on the preference list the patient is trying to be matched with
        while(noMatch):
            tentDoc = df[tentPatient][index] #Get the doctor that's at the index in the patient's list
            #if len(matches[tentDoc]) < doctorCapacities[tentDoc]:
            if len(matches[tentDoc]) < doctorCapacity: #If the doctor has spots open add patient to the list
                matches[tentDoc].add(tentPatient)
                noMatch = False
            else: #TO-DO
                for compareMatch in matches[tentDoc]: #If the current patient ranks the doctor higher than an exisiting match, replace the exisisting match
                #if index == patients[compareMatch].index(tentDoc):
                    if index < list(df[compareMatch]).index(tentDoc):
                        matches[tentDoc].remove(compareMatch)
                        q.append(compareMatch) #Add the existing patient back to the end of the queue
                        matches[tentDoc].add(tentPatient)
                        noMatch = False
                        break
            index += 1 #If a match hasn't been found go on to the next doctor on the list
    return matches

def calcCost(matches, patients): #Goes through the matches and totals up where each patient ranked their respective doctor, returns both the average and median ranking
    cost = 0
    costList = []
    for doctor in matches:
        for patient in matches[doctor]:
            cost += list(patients[patient]).index(doctor)
            costList.append(list(patients[patient]).index(doctor))
    return (cost,stats.median(costList))

  # print(matches)
  # print(calcCost(matches,df)[0]/patientNum)
  # print(calcCost(matches,df)[1])
